In [4]:
# ============================================================
# 1) SETUP & PATHS
# - Mount Google Drive
# - Create project folders
# - Define constants (densities, materials)
# ============================================================
import os, re, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 1.1) Mount Google Drive (Colab)
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print("Colab Drive mount skipped (running locally?):", e)

# 1.2) Project paths (edit if needed)
BASE_DIR = "/content/drive/MyDrive/PhotonAttenuation"
XCOM_DIR = os.path.join(BASE_DIR, "XCOM_data")    # <-- put your downloaded XCOM .txt/.dat files here
OUT_DIR  = os.path.join(BASE_DIR, "outputs")
FIG_DIR  = os.path.join(BASE_DIR, "figures")

for d in [BASE_DIR, XCOM_DIR, OUT_DIR, FIG_DIR]:
    os.makedirs(d, exist_ok=True)

# 1.3) Reference densities (g/cm^3) — NIST/ICRU typical values
rho_g_cm3 = {
    # --- 30 elements ---
    "H": 8.99e-5, "He": 1.785e-4, "C": 2.267, "N": 1.165e-3, "O": 1.331e-3,
    "Na": 0.971, "Mg": 1.738, "Al": 2.699, "Si": 2.33, "P": 2.20, "S": 2.07,
    "Cl": 3.214e-3, "K": 0.862, "Ca": 1.55, "Ti": 4.506, "V": 6.11, "Cr": 7.19,
    "Mn": 7.43, "Fe": 7.87, "Co": 8.90, "Ni": 8.908, "Cu": 8.96, "Zn": 7.14,
    "Zr": 6.52, "Mo": 10.28, "Ag": 10.49, "Sn": 7.31, "I": 4.93, "W": 19.3, "Pb": 11.34,
    # --- 10 compounds / materials ---
    "H2O": 1.00, "Air": 1.205e-3, "Concrete": 2.30, "HDPE": 0.95, "PMMA": 1.19,
    "PVC": 1.38, "Polycarbonate": 1.20, "BorosilicateGlass": 2.23,
    "ICRU_CorticalBone": 1.85, "ICRU_Adipose": 0.92
}

# 1.4) Materials lists
elements_list = [
    "H","He","C","N","O","Na","Mg","Al","Si","P","S","Cl","K","Ca","Ti","V",
    "Cr","Mn","Fe","Co","Ni","Cu","Zn","Zr","Mo","Ag","Sn","I","W","Pb"
]
compounds_list = [
    "H2O","Air","Concrete","HDPE","PMMA","PVC","Polycarbonate",
    "BorosilicateGlass","ICRU_CorticalBone","ICRU_Adipose"
]
all_40_materials = elements_list + compounds_list

print("✅ Paths ready.")
print("   XCOM_DIR:", XCOM_DIR)
print("   FIG_DIR :", FIG_DIR)
print("   OUT_DIR :", OUT_DIR)
print("   #materials:", len(all_40_materials))

Mounted at /content/drive
✅ Paths ready.
   XCOM_DIR: /content/drive/MyDrive/PhotonAttenuation/XCOM_data
   FIG_DIR : /content/drive/MyDrive/PhotonAttenuation/figures
   OUT_DIR : /content/drive/MyDrive/PhotonAttenuation/outputs
   #materials: 40


In [5]:
# ============================================================
# 2) READ XCOM FILES → MASTER DATAFRAME
# - Robust parser for NIST-like tables
# - Compute linear μ = (μ/ρ) * ρ
# - Save XCOM_master_dataset.csv
# ============================================================
def read_xcom_file(filepath, densities=rho_g_cm3):
    """
    Parse an XCOM text/dat file.
    Expected columns (full table):
      Energy  Coherent  Incoherent  Photoelectric  PairProd_Nuclear
      PairProd_Electron  Total_With_Coh  Total_Without_Coh
    Fallback: at least Energy and mu_over_rho.
    """
    name = os.path.splitext(os.path.basename(filepath))[0]
    name = name.replace("xcom_", "").replace("XCOM_", "")
    rho = densities.get(name, 1.0)

    cols_full = ["Energy_MeV","Coherent","Incoherent","Photoelectric",
                 "PairProd_Nuclear","PairProd_Electron","Total_With_Coh","Total_Without_Coh"]
    try:
        df = pd.read_csv(filepath, sep=r"\s+", skiprows=2, names=cols_full, engine="python")
        for c in cols_full:
            df[c] = pd.to_numeric(df[c], errors="coerce")
        df["mu_over_rho"] = df["Total_With_Coh"]
        df["PairProd_Total"] = (df["PairProd_Nuclear"].fillna(0)+df["PairProd_Electron"].fillna(0))
    except Exception:
        # fallback: lines with at least two numeric tokens (E, mu_over_rho)
        data = []
        with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
            for line in f:
                if not re.match(r"^\s*[-+0-9.]", line):
                    continue
                vals = re.findall(r"[-+]?\d*\.\d+|\d+", line)
                if len(vals) >= 2:
                    E = float(vals[0])
                    mu_rho = float(vals[1])
                    data.append((E, mu_rho))
        df = pd.DataFrame(data, columns=["Energy_MeV","mu_over_rho"])
        for c in ["Coherent","Incoherent","Photoelectric","PairProd_Nuclear","PairProd_Electron",
                  "Total_With_Coh","Total_Without_Coh","PairProd_Total"]:
            df[c] = np.nan
        df["Total_With_Coh"] = df["mu_over_rho"]

    df = df.dropna(subset=["Energy_MeV","mu_over_rho"]).copy()
    df["Material"] = name
    df["rho"] = rho
    df["mu_cm^-1"] = df["mu_over_rho"] * rho
    return df

# 2.1) Read all XCOM files from folder
frames = []
for fname in os.listdir(XCOM_DIR):
    if fname.lower().endswith((".txt",".dat")):
        path = os.path.join(XCOM_DIR, fname)
        try:
            frames.append(read_xcom_file(path))
            print(f"✅ Loaded: {fname}")
        except Exception as e:
            print(f"⚠️ Error in {fname} → {e}")

assert len(frames) > 0, "No XCOM files found in XCOM_DIR. Please download/export XCOM tables first."

master_df = pd.concat(frames, ignore_index=True).sort_values(["Material","Energy_MeV"]).reset_index(drop=True)
csv_master = os.path.join(OUT_DIR, "XCOM_master_dataset.csv")
master_df.to_csv(csv_master, index=False)
print("📁 Saved master dataset:", csv_master)
print(master_df.head(3))


✅ Loaded: xcom_Mg.txt
✅ Loaded: xcom_Na.txt
✅ Loaded: xcom_Bone.txt
✅ Loaded: xcom_O.txt
✅ Loaded: xcom_N.txt
✅ Loaded: xcom_P.txt
✅ Loaded: xcom_Cl.txt
✅ Loaded: xcom_Ca.txt
✅ Loaded: xcom_Polycarbonate.txt
✅ Loaded: xcom_Ag.txt
✅ Loaded: xcom_BorosilicateGlass.txt
✅ Loaded: xcom_C.txt
✅ Loaded: xcom_Ni.txt
✅ Loaded: xcom_Zn.txt
✅ Loaded: xcom_HDPE.txt
✅ Loaded: xcom_Zr.txt
✅ Loaded: xcom_ICRU_Adipose.txt
✅ Loaded: xcom_Concrete.txt
✅ Loaded: xcom_Cr.txt
✅ Loaded: xcom_Mn.txt
✅ Loaded: xcom_He.txt
✅ Loaded: xcom_PVC.txt
✅ Loaded: xcom_K.txt
✅ Loaded: xcom_Co.txt
✅ Loaded: xcom_Al.txt
✅ Loaded: xcom_I.txt
✅ Loaded: xcom_Si.txt
✅ Loaded: xcom_S.txt
✅ Loaded: xcom_V.txt
✅ Loaded: xcom_Sn.txt
✅ Loaded: xcom_W.txt
✅ Loaded: xcom_Fe.txt
✅ Loaded: xcom_H2O.txt
✅ Loaded: xcom_PMMA.txt
✅ Loaded: xcom_Mo.txt
✅ Loaded: xcom_Ti.txt
✅ Loaded: xcom_Pb.txt
✅ Loaded: xcom_H.txt
✅ Loaded: xcom_Air.txt
✅ Loaded: xcom_Cu.txt
📁 Saved master dataset: /content/drive/MyDrive/PhotonAttenuation/outputs/XCOM_m

In [6]:
# ============================================================
# 3) THREE-SLOPE PIECEWISE FIT
# - Fit log(μ) vs log(E) in 3 regions split by E1 and E2
# - Return slopes m_i and exponents c_i = -m_i
# - Batch run for elements (and optionally compounds)
# ============================================================
from dataclasses import dataclass
from typing import Optional, Dict, Any

@dataclass
class FitResult3:
    material: str
    E1: float
    E2: float
    m1: float; b1: float; c1: float
    m2: float; b2: float; c2: float
    m3: float; b3: float; c3: float
    RMSE_log: float

def _linfit(x, y):
    # simple least squares for y = m*x + b
    A = np.vstack([x, np.ones_like(x)]).T
    m, b = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, b

def fit_three_slope_for_material(df_mat: pd.DataFrame,
                                 E1_grid=None, E2_grid=None,
                                 min_pts=5) -> Optional[Dict[str,Any]]:
    """
    Fit log(μ) vs log(E) piecewise with 2 breakpoints (E1 < E2).
    """
    df = df_mat.dropna(subset=["mu_cm^-1","Energy_MeV"]).copy()
    if df.empty:
        return None

    E = df["Energy_MeV"].values
    mu = np.clip(df["mu_cm^-1"].values, 1e-30, None)
    x = np.log10(E)
    y = np.log10(mu)

    Emin, Emax = float(df["Energy_MeV"].min()), float(df["Energy_MeV"].max())
    if E1_grid is None:
        E1_grid = np.geomspace(max(0.02, Emin*1.2), min(0.45, Emax*0.6), 24)
    if E2_grid is None:
        # E2 should start >= 1.022 MeV if possible
        e2_min = max(1.022, Emin*1.5)
        e2_max = Emax * 0.98
        if e2_min >= e2_max:
            e2_min = min(max(1.022, Emin*1.1), Emax*0.95)
            e2_max = Emax * 0.99
        E2_grid = np.geomspace(e2_min, e2_max, 24)

    best = None
    best_err = np.inf

    for E1 in E1_grid:
        for E2 in E2_grid:
            if not (Emin < E1 < E2 < Emax):
                continue

            mask1 = E < E1
            mask2 = (E >= E1) & (E < E2)
            mask3 = E >= E2

            if (mask1.sum() < min_pts) or (mask2.sum() < min_pts) or (mask3.sum() < min_pts):
                continue

            m1, b1 = _linfit(x[mask1], y[mask1])
            m2, b2 = _linfit(x[mask2], y[mask2])
            m3, b3 = _linfit(x[mask3], y[mask3])

            y_hat = np.empty_like(y)
            y_hat[mask1] = m1*x[mask1] + b1
            y_hat[mask2] = m2*x[mask2] + b2
            y_hat[mask3] = m3*x[mask3] + b3

            rmse = float(np.sqrt(np.nanmean((y - y_hat)**2)))
            if rmse < best_err:
                best_err = rmse
                best = dict(E1=float(E1), E2=float(E2),
                            m1=float(m1), b1=float(b1), c1=float(-m1),
                            m2=float(m2), b2=float(b2), c2=float(-m2),
                            m3=float(m3), b3=float(b3), c3=float(-m3),
                            RMSE_log=rmse)
    return best

def plot_three_slope(df_mat: pd.DataFrame, fitres: Dict[str,Any], title=None, savepath=None):
    E = df_mat["Energy_MeV"].values
    mu = np.clip(df_mat["mu_cm^-1"].values, 1e-30, None)
    x = np.log10(E); y = np.log10(mu)

    E1, E2 = fitres["E1"], fitres["E2"]
    m1,b1 = fitres["m1"], fitres["b1"]
    m2,b2 = fitres["m2"], fitres["b2"]
    m3,b3 = fitres["m3"], fitres["b3"]

    plt.figure(figsize=(7,5))
    plt.scatter(x, y, s=12, alpha=0.65, label="XCOM (log-log)")

    # plot segments
    xs = np.linspace(x.min(), x.max(), 400)
    Es = 10**xs
    yhat = np.piecewise(
        xs,
        [Es < E1, (Es >= E1) & (Es < E2), Es >= E2],
        [lambda z: m1*z + b1, lambda z: m2*z + b2, lambda z: m3*z + b3]
    )
    plt.plot(xs, yhat, lw=2.5, label="3-slope fit")

    # vertical lines at E1,E2
    plt.axvline(np.log10(E1), ls="--", lw=1, color="k")
    plt.axvline(np.log10(E2), ls="--", lw=1, color="k")

    plt.xlabel("log10(Energy [MeV])")
    plt.ylabel("log10(μ [cm^-1])")
    plt.title(title or "Three-Slope Piecewise Fit")
    plt.grid(True, ls=":", alpha=0.5)
    plt.legend()
    if savepath:
        plt.savefig(savepath, dpi=220, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

# 3.1) Demo on Pb (lead) if available
if (master_df["Material"] == "Pb").any():
    df_pb = master_df[master_df["Material"]=="Pb"]
    fit_pb = fit_three_slope_for_material(df_pb, min_pts=5)
    if fit_pb:
        print("Pb →", fit_pb)
        plot_three_slope(df_pb, fit_pb, title="3-Slope Fit — Pb",
                         savepath=os.path.join(FIG_DIR, "Pb_three_slope.png"))

# 3.2) Batch on 30 elements
results = []
for mat in elements_list:
    df_mat = master_df[master_df["Material"]==mat]
    if df_mat.empty:
        print(f"⚠️ Missing data for {mat}")
        continue
    fitres = fit_three_slope_for_material(df_mat, min_pts=5)
    if fitres is None:
        print(f"❌ Could not fit 3-slope for {mat}")
        continue
    fitres["material"] = mat
    results.append(fitres)
    plot_three_slope(df_mat, fitres, title=f"3-Slope Fit — {mat}",
                     savepath=os.path.join(FIG_DIR, f"{mat}_three_slope.png"))
    print(f"✅ {mat}: E1={fitres['E1']:.4g}, E2={fitres['E2']:.4g}, RMSE_log={fitres['RMSE_log']:.3f}")

df_res = pd.DataFrame(results).sort_values("material").reset_index(drop=True)
csv_res = os.path.join(OUT_DIR, "XCOM_three_slope_RESULTS_elements.csv")
df_res.to_csv(csv_res, index=False)
print("📁 Saved elements 3-slope results:", csv_res)


Pb → {'E1': 0.02289921211223899, 'E2': 1.022, 'm1': -1.4771013220103346, 'b1': 0.47162167770667984, 'c1': 1.4771013220103346, 'm2': -1.7065691396964053, 'b2': -0.18172653624379675, 'c2': 1.7065691396964053, 'm3': 0.10316682390984448, 'b3': -0.27616823377480626, 'c3': -0.10316682390984448, 'RMSE_log': 0.12240229996568751}
✅ H: E1=0.02, E2=149.8, RMSE_log=0.090
✅ He: E1=0.02, E2=55.24, RMSE_log=0.086
✅ C: E1=0.0229, E2=20.37, RMSE_log=0.027
✅ N: E1=0.0229, E2=20.37, RMSE_log=0.022
✅ O: E1=0.0229, E2=12.37, RMSE_log=0.024
✅ Na: E1=0.0229, E2=7.514, RMSE_log=0.163
✅ Mg: E1=0.03002, E2=7.514, RMSE_log=0.161
✅ Al: E1=0.03002, E2=7.514, RMSE_log=0.153
✅ Si: E1=0.03002, E2=7.514, RMSE_log=0.151
✅ P: E1=0.03002, E2=4.563, RMSE_log=0.150
✅ S: E1=0.03002, E2=4.563, RMSE_log=0.149
✅ Cl: E1=0.04506, E2=4.563, RMSE_log=0.149
✅ K: E1=0.04506, E2=4.563, RMSE_log=0.147
✅ Ca: E1=0.06763, E2=4.563, RMSE_log=0.144
✅ Ti: E1=0.06763, E2=4.563, RMSE_log=0.142
✅ V: E1=0.1524, E2=4.563, RMSE_log=0.138
✅ Cr: E1

In [7]:
# ============================================================
# 3) THREE-SLOPE PIECEWISE FIT
# - Fit log(μ) vs log(E) in 3 regions split by E1 and E2
# - Return slopes m_i and exponents c_i = -m_i
# - Batch run for elements (and optionally compounds)
# ============================================================
from dataclasses import dataclass
from typing import Optional, Dict, Any

@dataclass
class FitResult3:
    material: str
    E1: float
    E2: float
    m1: float; b1: float; c1: float
    m2: float; b2: float; c2: float
    m3: float; b3: float; c3: float
    RMSE_log: float

def _linfit(x, y):
    # simple least squares for y = m*x + b
    A = np.vstack([x, np.ones_like(x)]).T
    m, b = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, b

def fit_three_slope_for_material(df_mat: pd.DataFrame,
                                 E1_grid=None, E2_grid=None,
                                 min_pts=5) -> Optional[Dict[str,Any]]:
    """
    Fit log(μ) vs log(E) piecewise with 2 breakpoints (E1 < E2).
    """
    df = df_mat.dropna(subset=["mu_cm^-1","Energy_MeV"]).copy()
    if df.empty:
        return None

    E = df["Energy_MeV"].values
    mu = np.clip(df["mu_cm^-1"].values, 1e-30, None)
    x = np.log10(E)
    y = np.log10(mu)

    Emin, Emax = float(df["Energy_MeV"].min()), float(df["Energy_MeV"].max())
    if E1_grid is None:
        E1_grid = np.geomspace(max(0.02, Emin*1.2), min(0.45, Emax*0.6), 24)
    if E2_grid is None:
        # E2 should start >= 1.022 MeV if possible
        e2_min = max(1.022, Emin*1.5)
        e2_max = Emax * 0.98
        if e2_min >= e2_max:
            e2_min = min(max(1.022, Emin*1.1), Emax*0.95)
            e2_max = Emax * 0.99
        E2_grid = np.geomspace(e2_min, e2_max, 24)

    best = None
    best_err = np.inf

    for E1 in E1_grid:
        for E2 in E2_grid:
            if not (Emin < E1 < E2 < Emax):
                continue

            mask1 = E < E1
            mask2 = (E >= E1) & (E < E2)
            mask3 = E >= E2

            if (mask1.sum() < min_pts) or (mask2.sum() < min_pts) or (mask3.sum() < min_pts):
                continue

            m1, b1 = _linfit(x[mask1], y[mask1])
            m2, b2 = _linfit(x[mask2], y[mask2])
            m3, b3 = _linfit(x[mask3], y[mask3])

            y_hat = np.empty_like(y)
            y_hat[mask1] = m1*x[mask1] + b1
            y_hat[mask2] = m2*x[mask2] + b2
            y_hat[mask3] = m3*x[mask3] + b3

            rmse = float(np.sqrt(np.nanmean((y - y_hat)**2)))
            if rmse < best_err:
                best_err = rmse
                best = dict(E1=float(E1), E2=float(E2),
                            m1=float(m1), b1=float(b1), c1=float(-m1),
                            m2=float(m2), b2=float(b2), c2=float(-m2),
                            m3=float(m3), b3=float(b3), c3=float(-m3),
                            RMSE_log=rmse)
    return best

def plot_three_slope(df_mat: pd.DataFrame, fitres: Dict[str,Any], title=None, savepath=None):
    E = df_mat["Energy_MeV"].values
    mu = np.clip(df_mat["mu_cm^-1"].values, 1e-30, None)
    x = np.log10(E); y = np.log10(mu)

    E1, E2 = fitres["E1"], fitres["E2"]
    m1,b1 = fitres["m1"], fitres["b1"]
    m2,b2 = fitres["m2"], fitres["b2"]
    m3,b3 = fitres["m3"], fitres["b3"]

    plt.figure(figsize=(7,5))
    plt.scatter(x, y, s=12, alpha=0.65, label="XCOM (log-log)")

    # plot segments
    xs = np.linspace(x.min(), x.max(), 400)
    Es = 10**xs
    yhat = np.piecewise(
        xs,
        [Es < E1, (Es >= E1) & (Es < E2), Es >= E2],
        [lambda z: m1*z + b1, lambda z: m2*z + b2, lambda z: m3*z + b3]
    )
    plt.plot(xs, yhat, lw=2.5, label="3-slope fit")

    # vertical lines at E1,E2
    plt.axvline(np.log10(E1), ls="--", lw=1, color="k")
    plt.axvline(np.log10(E2), ls="--", lw=1, color="k")

    plt.xlabel("log10(Energy [MeV])")
    plt.ylabel("log10(μ [cm^-1])")
    plt.title(title or "Three-Slope Piecewise Fit")
    plt.grid(True, ls=":", alpha=0.5)
    plt.legend()
    if savepath:
        plt.savefig(savepath, dpi=220, bbox_inches="tight")
        plt.close()
    else:
        plt.show()

# 3.1) Demo on Pb (lead) if available
if (master_df["Material"] == "Pb").any():
    df_pb = master_df[master_df["Material"]=="Pb"]
    fit_pb = fit_three_slope_for_material(df_pb, min_pts=5)
    if fit_pb:
        print("Pb →", fit_pb)
        plot_three_slope(df_pb, fit_pb, title="3-Slope Fit — Pb",
                         savepath=os.path.join(FIG_DIR, "Pb_three_slope.png"))

# 3.2) Batch on 30 elements
results = []
for mat in elements_list:
    df_mat = master_df[master_df["Material"]==mat]
    if df_mat.empty:
        print(f"⚠️ Missing data for {mat}")
        continue
    fitres = fit_three_slope_for_material(df_mat, min_pts=5)
    if fitres is None:
        print(f"❌ Could not fit 3-slope for {mat}")
        continue
    fitres["material"] = mat
    results.append(fitres)
    plot_three_slope(df_mat, fitres, title=f"3-Slope Fit — {mat}",
                     savepath=os.path.join(FIG_DIR, f"{mat}_three_slope.png"))
    print(f"✅ {mat}: E1={fitres['E1']:.4g}, E2={fitres['E2']:.4g}, RMSE_log={fitres['RMSE_log']:.3f}")

df_res = pd.DataFrame(results).sort_values("material").reset_index(drop=True)
csv_res = os.path.join(OUT_DIR, "XCOM_three_slope_RESULTS_elements.csv")
df_res.to_csv(csv_res, index=False)
print("📁 Saved elements 3-slope results:", csv_res)


Pb → {'E1': 0.02289921211223899, 'E2': 1.022, 'm1': -1.4771013220103346, 'b1': 0.47162167770667984, 'c1': 1.4771013220103346, 'm2': -1.7065691396964053, 'b2': -0.18172653624379675, 'c2': 1.7065691396964053, 'm3': 0.10316682390984448, 'b3': -0.27616823377480626, 'c3': -0.10316682390984448, 'RMSE_log': 0.12240229996568751}
✅ H: E1=0.02, E2=149.8, RMSE_log=0.090
✅ He: E1=0.02, E2=55.24, RMSE_log=0.086
✅ C: E1=0.0229, E2=20.37, RMSE_log=0.027
✅ N: E1=0.0229, E2=20.37, RMSE_log=0.022
✅ O: E1=0.0229, E2=12.37, RMSE_log=0.024
✅ Na: E1=0.0229, E2=7.514, RMSE_log=0.163
✅ Mg: E1=0.03002, E2=7.514, RMSE_log=0.161
✅ Al: E1=0.03002, E2=7.514, RMSE_log=0.153
✅ Si: E1=0.03002, E2=7.514, RMSE_log=0.151
✅ P: E1=0.03002, E2=4.563, RMSE_log=0.150
✅ S: E1=0.03002, E2=4.563, RMSE_log=0.149
✅ Cl: E1=0.04506, E2=4.563, RMSE_log=0.149
✅ K: E1=0.04506, E2=4.563, RMSE_log=0.147
✅ Ca: E1=0.06763, E2=4.563, RMSE_log=0.144
✅ Ti: E1=0.06763, E2=4.563, RMSE_log=0.142
✅ V: E1=0.1524, E2=4.563, RMSE_log=0.138
✅ Cr: E1

In [8]:
# ============================================================
# 4) CORRELATION ANALYSIS: c1, c2, c3 vs log10(Z_eff)
# - Define Z for elements
# - Approximate Z_eff for compounds (simple stoichiometric or literature values)
# - Linear regression and plots
# ============================================================
# 4.1) Atomic numbers for elements
Z_number = {
    "H":1,"He":2,"C":6,"N":7,"O":8,"Na":11,"Mg":12,"Al":13,"Si":14,"P":15,"S":16,
    "Cl":17,"K":19,"Ca":20,"Ti":22,"V":23,"Cr":24,"Mn":25,"Fe":26,"Co":27,"Ni":28,
    "Cu":29,"Zn":30,"Zr":40,"Mo":42,"Ag":47,"Sn":50,"I":53,"W":74,"Pb":82
}

# 4.2) Approximate Z_eff for 10 compounds (heuristic; acceptable for correlation sanity-checks)
Z_eff_map = {
    "H2O": 7.42,            # typical literature
    "Air": 7.30,            # dry air approximation
    "Concrete": 11.50,      # typical ordinary concrete
    "HDPE": 5.74,           # CH2 polymer
    "PMMA": 6.50,           # C5H8O2 ~ acrylic
    "PVC": 14.00,           # C2H3Cl (Cl raises Z_eff)
    "Polycarbonate": 6.80,  # approx.
    "BorosilicateGlass": 10.80, # approx.
    "ICRU_CorticalBone": 13.80, # ICRU reported ~13–14
    "ICRU_Adipose": 5.92
}

# 4.3) Build Z_eff for the elements
Zeff_series = []
for m in df_res["material"]:
    Zeff = Z_number.get(m, None)
    Zeff_series.append(Zeff)
df_res["Zeff"] = Zeff_series

# 4.4) (Optional) If you also fit compounds, load them and append with Z_eff_map
# Example skeleton (uncomment if you have compounds CSV):
# comp_csv = os.path.join(OUT_DIR, "XCOM_three_slope_RESULTS_compounds.csv")
# if os.path.exists(comp_csv):
#     df_comp = pd.read_csv(comp_csv)
#     df_comp["Zeff"] = df_comp["material"].map(Z_eff_map)
#     df_all = pd.concat([df_res, df_comp], ignore_index=True)
# else:
#     df_all = df_res.copy()

df_all = df_res.copy()  # here we use elements-only unless you add compounds
df_all = df_all.dropna(subset=["Zeff"]).copy()
df_all["logZ"] = np.log10(df_all["Zeff"])

def lin_reg(x, y):
    A = np.vstack([x, np.ones_like(x)]).T
    m, b = np.linalg.lstsq(A, y, rcond=None)[0]
    yhat = m*x + b
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    r2 = 1 - ss_res/ss_tot if ss_tot > 0 else np.nan
    return m, b, r2

# 4.5) Fit c1, c2, c3 vs logZ
summ = []
for ci in ["c1","c2","c3"]:
    m, b, r2 = lin_reg(df_all["logZ"].values, df_all[ci].values)
    summ.append({"coef":ci, "slope":m, "intercept":b, "R2":r2})
    # plot
    xs = np.linspace(df_all["logZ"].min(), df_all["logZ"].max(), 200)
    ys = m*xs + b
    plt.figure(figsize=(6,4.3))
    plt.scatter(df_all["logZ"], df_all[ci], s=16, alpha=0.7, label="Elements")
    plt.plot(xs, ys, lw=2, label=f"Fit: y={m:.3f}·logZ+{b:.3f} (R²={r2:.3f})")
    plt.xlabel("log10(Z_eff)")
    plt.ylabel(ci)
    plt.grid(True, ls=":", alpha=0.5)
    plt.legend()
    outp = os.path.join(FIG_DIR, f"corr_{ci}.png")
    plt.savefig(outp, dpi=220, bbox_inches="tight")
    plt.close()
    print(f"📈 Saved correlation plot for {ci} → {outp}")

df_corr = pd.DataFrame(summ)
csv_corr = os.path.join(OUT_DIR, "correlation_summary_c123_vs_logZ.csv")
df_corr.to_csv(csv_corr, index=False)
print("📁 Saved correlation summary:", csv_corr)
print(df_corr)


📈 Saved correlation plot for c1 → /content/drive/MyDrive/PhotonAttenuation/figures/corr_c1.png
📈 Saved correlation plot for c2 → /content/drive/MyDrive/PhotonAttenuation/figures/corr_c2.png
📈 Saved correlation plot for c3 → /content/drive/MyDrive/PhotonAttenuation/figures/corr_c3.png
📁 Saved correlation summary: /content/drive/MyDrive/PhotonAttenuation/outputs/correlation_summary_c123_vs_logZ.csv
  coef     slope  intercept        R2
0   c1 -0.228405   2.335004  0.058989
1   c2  0.510180   0.029741  0.406642
2   c3 -0.058760   0.009222  0.912910


In [10]:
# ============================================================
# 5) PHYSICAL TIME CALIBRATION (Cs-137, Co-60)
# - λ = ln(2)/T_half
# - Interpolate μ(E) from master_df (log-log)
# - Compute I/I0 over time and thickness
# - Save plots + CSV summary
# ============================================================
def lambda_from_half_life(T_half_years, unit="days"):
    ln2 = np.log(2.0)
    if unit == "days":
        return ln2 / (T_half_years * 365.25)
    elif unit == "years":
        return ln2 / T_half_years
    else:
        raise ValueError("unit must be 'days' or 'years'")

def mu_interp(material, E_MeV, df=master_df):
    d = df[(df["Material"]==material)].dropna(subset=["Energy_MeV","mu_cm^-1"])
    if d.empty:
        raise ValueError(f"No XCOM data for {material}")
    x = np.log(d["Energy_MeV"].values)
    y = np.log(np.clip(d["mu_cm^-1"].values, 1e-30, None))
    E = float(np.clip(E_MeV, d["Energy_MeV"].min(), d["Energy_MeV"].max()))
    yhat = np.interp(np.log(E), x, y)
    return float(np.exp(yhat))

def transmission_spatial(material, x_cm, lines):
    # Sum of line transmissions weighted by yield
    tot = 0.0
    for line in lines:
        mu = mu_interp(material, line["E_MeV"])
        tot += line["yield"] * np.exp(-mu * x_cm)
    return tot

def transmission_spatio_temporal(material, x_cm, lines, lam_day, t_days):
    return transmission_spatial(material, x_cm, lines) * np.exp(-lam_day * t_days)

sources = {
    "Cs-137": {
        "half_life_years": 30.17,
        "lines": [{"E_MeV":0.661657, "yield":0.851}]
    },
    "Co-60": {
        "half_life_years": 5.27,
        "lines": [{"E_MeV":1.173228, "yield":0.9985},
                  {"E_MeV":1.332492, "yield":0.9998}]
    }
}

materials_for_time = ["Pb","Fe","Concrete","Al","H2O","W"]
thicknesses_cm = [0.0, 0.5, 1.0, 2.0, 5.0]
t_years = np.linspace(0, 10, 300)
t_days = t_years * 365.25

rows = []
for src, meta in sources.items():
    lam_d = lambda_from_half_life(meta["half_life_years"], "days")
    for mat in materials_for_time:
        for x_cm in thicknesses_cm:
            Ivals = [transmission_spatio_temporal(mat, x_cm, meta["lines"], lam_d, td) for td in t_days]
            # plots per material
            plt.figure(figsize=(7,4.8))
            for xx in thicknesses_cm:
                Iy = [transmission_spatio_temporal(mat, xx, meta["lines"], lam_d, td) for td in t_days]
                plt.plot(t_years, Iy, lw=2, label=f"{xx} cm")
            plt.yscale("log")
            plt.xlabel("Time (years)")
            plt.ylabel("I / I0 (log scale)")
            plt.title(f"{src} — {mat}")
            plt.grid(True, ls=":", alpha=0.5)
            plt.legend(title="Thickness")
            fname = f"{src.replace('-','_')}_{mat}_decay_curves.png"
            plt.savefig(os.path.join(FIG_DIR, fname), dpi=230, bbox_inches="tight")
            plt.close()

            rows.append({
                "source": src,
                "material": mat,
                "thickness_cm": x_cm,
                "lambda_day^-1": lam_d,
                "I_over_I0_at_1y": transmission_spatio_temporal(mat, x_cm, meta["lines"], lam_d, 365.25),
                "I_over_I0_at_5y": transmission_spatio_temporal(mat, x_cm, meta["lines"], lam_d, 5*365.25),
                "I_over_I0_at_10y": transmission_spatio_temporal(mat, x_cm, meta["lines"], lam_d, 10*365.25)
            })

# Fixed-time material comparison at 5 years (optional illustrative plots)
t_fixed_days = 5*365.25
for src, meta in sources.items():
    lam_d = lambda_from_half_life(meta["half_life_years"], "days")
    xs = np.linspace(0, 5, 80)
    plt.figure(figsize=(7,4.8))
    for mat in ["Pb","Fe","Concrete","H2O"]:
        Iy = [transmission_spatio_temporal(mat, x, meta["lines"], lam_d, t_fixed_days) for x in xs]
        plt.plot(xs, Iy, lw=2, label=mat)
    plt.yscale("log")
    plt.xlabel("Thickness (cm)")
    plt.ylabel("I / I0 at 5 years (log scale)")
    plt.title(f"{src} — Material Comparison at 5 years")
    plt.grid(True, ls=":", alpha=0.5)
    fname = f"{src.replace('-','_')}_MaterialComparison_5years.png"
    plt.legend()
    plt.savefig(os.path.join(FIG_DIR, fname), dpi=230, bbox_inches="tight")
    plt.close()

df_time = pd.DataFrame(rows)
csv_time = os.path.join(OUT_DIR, "time_calibration_summary.csv")
df_time.to_csv(csv_time, index=False)
print("📁 Saved time-calibration summary:", csv_time)
print(df_time.head(8))


📁 Saved time-calibration summary: /content/drive/MyDrive/PhotonAttenuation/outputs/time_calibration_summary.csv
   source material  thickness_cm  lambda_day^-1  I_over_I0_at_1y  \
0  Cs-137       Pb           0.0       0.000063         0.831671   
1  Cs-137       Pb           0.5       0.000063         0.442919   
2  Cs-137       Pb           1.0       0.000063         0.235883   
3  Cs-137       Pb           2.0       0.000063         0.066902   
4  Cs-137       Pb           5.0       0.000063         0.001526   
5  Cs-137       Fe           0.0       0.000063         0.831671   
6  Cs-137       Fe           0.5       0.000063         0.622882   
7  Cs-137       Fe           1.0       0.000063         0.466508   

   I_over_I0_at_5y  I_over_I0_at_10y  
0         0.758648          0.676319  
1         0.404029          0.360184  
2         0.215172          0.191821  
3         0.061028          0.054405  
4         0.001392          0.001241  
5         0.758648          0.676319  
6 

In [11]:
# ============================================================
# 6) β-SENSITIVITY — NON-SEPARABLE MODEL
# - I = I0 * exp[-μ (x + β t)]
# - Sweep β and compute t_half_eff = ln(2)/(μ β)
# ============================================================
E_demo = 0.662   # MeV
materials_beta = ["Pb","Fe","Concrete"]
beta_values = np.linspace(0.0, 0.10, 6)  # cm/year
t_years = np.linspace(0, 20, 400)
x_fixed = 1.0  # cm

# Plots for each material
for mat in materials_beta:
    mu = mu_interp(mat, E_demo)
    plt.figure(figsize=(7,5))
    for beta in beta_values:
        Ivals = np.exp(-mu * (x_fixed + beta*t_years))
        plt.plot(t_years, Ivals, lw=2, label=f"β={beta:.02f} cm/yr")
    plt.yscale("log")
    plt.xlabel("Time (years)")
    plt.ylabel("I / I0 (log scale)")
    plt.title(f"β Sensitivity — {mat} at {E_demo:.3f} MeV, x={x_fixed} cm")
    plt.grid(True, ls=":", alpha=0.5)
    plt.legend(title="Degradation rate β")
    outp = os.path.join(FIG_DIR, f"{mat}_beta_sensitivity.png")
    plt.savefig(outp, dpi=230, bbox_inches="tight")
    plt.close()
    print("🖼️ Saved:", outp)

# Table of effective temporal half-value times
rows = []
for mat in materials_beta:
    mu = mu_interp(mat, E_demo)
    for beta in beta_values[1:]:  # skip 0 to avoid division by zero
        t_half_eff = math.log(2)/(mu*beta)
        rows.append({"material":mat, "beta_cm_per_year":beta, "mu_cm^-1":mu, "t_half_eff_years":t_half_eff})
df_beta = pd.DataFrame(rows)
csv_beta = os.path.join(OUT_DIR, "beta_sensitivity_summary.csv")
df_beta.to_csv(csv_beta, index=False)
print("📁 Saved beta-sensitivity summary:", csv_beta)
print(df_beta.head(10))


🖼️ Saved: /content/drive/MyDrive/PhotonAttenuation/figures/Pb_beta_sensitivity.png
🖼️ Saved: /content/drive/MyDrive/PhotonAttenuation/figures/Fe_beta_sensitivity.png
🖼️ Saved: /content/drive/MyDrive/PhotonAttenuation/figures/Concrete_beta_sensitivity.png
📁 Saved beta-sensitivity summary: /content/drive/MyDrive/PhotonAttenuation/outputs/beta_sensitivity_summary.csv
  material  beta_cm_per_year  mu_cm^-1  t_half_eff_years
0       Pb              0.02  1.259327         27.520534
1       Pb              0.04  1.259327         13.760267
2       Pb              0.06  1.259327          9.173511
3       Pb              0.08  1.259327          6.880134
4       Pb              0.10  1.259327          5.504107
5       Fe              0.02  0.578017         59.959109
6       Fe              0.04  0.578017         29.979555
7       Fe              0.06  0.578017         19.986370
8       Fe              0.08  0.578017         14.989777
9       Fe              0.10  0.578017         11.991822


In [14]:
# ============================================================
# 7) MATERIAL AGING VISUALIZATION
# - μ(t) = μ0 * e^(-λ t), I(t) = exp[-μ(t) x]
# - Demonstration (Pb, 0.662 MeV)
# ============================================================
mu0 = mu_interp("Pb", 0.662)  # ~1.42 cm^-1 typically at 0.662 MeV
lam_year = 0.02               # example material-degradation rate per year
x_cm = 1.0
t_years = np.linspace(0, 50, 400)

mu_t = mu0 * np.exp(-lam_year * t_years)
I_t  = np.exp(-mu_t * x_cm)

fig, ax1 = plt.subplots(figsize=(7,5))

# Plot μ(t)
ax1.plot(t_years, mu_t, color="tab:blue", lw=2.5, label=r"$\mu(t)$")
ax1.set_xlabel("Time (years)")
ax1.set_ylabel(r"$\mu(t)$ [cm$^{-1}$]", color="tab:blue")
ax1.tick_params(axis='y', labelcolor="tab:blue")
ax1.grid(True, ls=":", alpha=0.5)

# Plot I/I0 on secondary axis
ax2 = ax1.twinx()
ax2.plot(t_years, I_t, color="tab:red", lw=2.5, label=r"$I/I_0$")
ax2.set_ylabel(r"$I/I_0$", color="tab:red")
ax2.tick_params(axis='y', labelcolor="tab:red")
ax2.set_yscale("log")

plt.title("Material Aging and Degradation — Lead (Pb)")
fig.tight_layout()
outp = os.path.join(FIG_DIR, "material_aging_placeholder.png")
plt.savefig(outp, dpi=230, bbox_inches="tight")
plt.close()
print("🖼️ Saved:", outp)



🖼️ Saved: /content/drive/MyDrive/PhotonAttenuation/figures/material_aging_placeholder.png


In [13]:
# ============================================================
# 8) UNIFIED EMPIRICAL–TEMPORAL SCHEMATIC
# - Conceptual flow: XCOM → Empirical → Piecewise → Temporal → Unified
# ============================================================
from matplotlib.patches import FancyBboxPatch

fig, ax = plt.subplots(figsize=(9,7))
ax.axis("off")

def box(text, xy, color="#E0E0E0", fontsize=11):
    ax.add_patch(FancyBboxPatch((xy[0]-0.4, xy[1]-0.2), 0.8, 0.4,
                                boxstyle="round,pad=0.02", fc=color, ec="black", lw=1.1))
    ax.text(xy[0], xy[1], text, ha="center", va="center", fontsize=fontsize, wrap=True)

def arrow(y1, y2):
    ax.arrow(0, y1-0.2, 0, y2-(y1-0.2)-0.4, head_width=0.05, head_length=0.1, fc="black", ec="black")

box("XCOM Reference Data\n(μ/ρ vs E)", (0, 4.0), "#B3E5FC", 12)
arrow(3.8, 3.2)
box("Global Log–Linear Model\nμ=a·Z^b·ρ^d·E^{−c}", (0, 3.0), "#C8E6C9", 11)
arrow(2.8, 2.2)
box("Piecewise Models\nTwo-Slope: {c1,c2}\nThree-Slope: {c1,c2,c3}", (0, 2.0), "#FFF9C4", 11)
arrow(1.8, 1.2)
box("Temporal Models\nSeparable: I=I0·e^{−μx−λt}\nNon-Separable: I=I0·e^{−μ(x+βt)}", (0, 1.0), "#FFE0B2", 11)
arrow(0.8, 0.2)
box("Unified Empirical–Temporal\nI=I0·e^{−aZ^bρ^dE^{−c}(x+βt)}·e^{−λt}", (0, 0.0), "#D1C4E9", 10)

plt.title("Unified Empirical–Temporal Attenuation Framework", fontsize=13, pad=20)
outp = os.path.join(FIG_DIR, "unified_framework_schematic.png")
plt.savefig(outp, dpi=300, bbox_inches="tight")
plt.close()
print("🖼️ Saved:", outp)

print("\n✅ All sections executed. Figures and CSV files are under:")
print("   FIG_DIR:", FIG_DIR)
print("   OUT_DIR:", OUT_DIR)
print("   Master CSV:", os.path.join(OUT_DIR, 'XCOM_master_dataset.csv'))


🖼️ Saved: /content/drive/MyDrive/PhotonAttenuation/figures/unified_framework_schematic.png

✅ All sections executed. Figures and CSV files are under:
   FIG_DIR: /content/drive/MyDrive/PhotonAttenuation/figures
   OUT_DIR: /content/drive/MyDrive/PhotonAttenuation/outputs
   Master CSV: /content/drive/MyDrive/PhotonAttenuation/outputs/XCOM_master_dataset.csv
